In [65]:
import os
data_dir = 'C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter08/data'
input_dir = os.path.join(data_dir, '0_input')
output_dir = os.path.join(data_dir, '1_output')
master_dir = os.path.join(data_dir, '99_master')
model_dir = 'models'
os.makedirs(input_dir, exist_ok = True)
os.makedirs(output_dir, exist_ok = True)
os.makedirs(master_dir, exist_ok = True)
os.makedirs(model_dir, exist_ok = True)

In [66]:
import pandas as pd
m_area_file = 'C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter08/data/99_master/m_area.csv'
m_store_file = 'C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter08/data/99_master/m_store.csv'
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))

In [67]:
tg_ym = '202103'
target_file = 'tbl_order_' + tg_ym + '.csv'
target_data = pd.read_csv(os.path.join(input_dir, target_file))

import datetime

max_date = pd.to_datetime(target_data['order_accept_date']).max()
min_date = pd.to_datetime(target_data['order_accept_date']).min()
max_str_date = max_date.strftime('%Y%m')
min_str_date = min_date.strftime('%Y%m')

if tg_ym == min_str_date and tg_ym == max_str_date:
    print('날짜가 일치합니다')
else:
    raise Exception('날짜가 일치하지 않습니다')

날짜가 일치합니다


In [68]:
def calc_delta(t):
    t1, t2, = t
    delta = t2 - t1
    return delta.total_seconds() / 60

# 데이터 가공 및 매장별 집계 함수로 실행
def data_processing(order_data):
    order_data = order_data.loc[order_data['store_id'] != 990]
    order_data = pd.merge(order_data, m_store, on = 'store_id', how = 'left')
    order_data = pd.merge(order_data, m_area, on = 'area_cd', how = 'left')
    order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'delivery'
    order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'takeout'
    order_data.loc[order_data['status'] == 0, 'status_name'] = '주문 접수'
    order_data.loc[order_data['status'] == 1, 'status_name'] = '계산 완료'
    order_data.loc[order_data['status'] == 2, 'status_name'] = '배달 완료'
    order_data.loc[order_data['status'] == 9, 'status_name'] = '주문 취소'
    
    order_data.loc[:, 'order_accept_datetime'] = pd.to_datetime(order_data['order_accept_date'])
    order_data.loc[:, 'delivered_datetime'] = pd.to_datetime(order_data['delivered_date'])
    order_data.loc[:, 'delta'] = order_data[['order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis = 1)
    
    order_data.loc[:, 'order_accept_hour'] = order_data['order_accept_datetime'].dt.hour
    order_data.loc[:, 'order_accept_weekday'] = order_data['order_accept_datetime'].dt.weekday
    order_data.loc[order_data['order_accept_weekday'] >= 5, 'weekday_info'] = '휴일'
    order_data.loc[order_data['order_accept_weekday'] < 5, 'weekday_info'] = '평일'
    
    store_data = order_data.groupby(['store_name']).count()[['order_id']]
    store_f = order_data.loc[
        (order_data['status_name'] == '배달 완료') | 
        (order_data['status_name'] == '결제 완료')
        ].groupby(['store_name']).count()[['order_id']]
    store_c = order_data.loc[
        order_data['status_name'] == '주문 취소'
        ].groupby(['store_name']).count()[['order_id']]
    store_d = order_data.loc[
        order_data['takeout_name'] == 'delivery'
        ].groupby(['store_name']).count()[['order_id']]
    store_t = order_data.loc[
        order_data['takeout_name'] == 'takeout'
        ].groupby(['store_name']).count()[['order_id']]

    store_weekday = order_data.loc[
        order_data['weekday_info'] == '평일'
        ].groupby(['store_name']).count()[['order_id']]
    store_weekend = order_data.loc[
        order_data['weekday_info'] == '휴일'
        ].groupby(['store_name']).count()[['order_id']]
    
    times = order_data['order_accept_hour'].unique()
    store_time = []
    
    for time in times:
        time_tmp = order_data.loc[
            order_data['order_accept_hour'] == time
            ].groupby(['store_name']).count()[['order_id']]
        time_tmp.columns = [f'order_time_{time}']
        store_time.append(time_tmp)
    store_time = pd.concat(store_time, axis = 1)
    
    store_delta = order_data.loc[
        order_data['status_name'] != '주문 취소'
        ].groupby(['store_name']).mean()[['delta']]
    store_data.columns = ['order']
    store_f.columns = ['order_fin']
    store_c.columns = ['order_cancel']
    store_d.columns = ['order_delivery']
    store_t.columns = ['order_takeout']

    store_weekday.columns = ['order_weekday']
    store_weekend.columns = ['order_weekend']
    store_delta.columns = ['delta_avg']
    store_data = pd.concat([
        store_data, store_f, store_c, store_d, store_t,
        store_weekday, store_weekend, store_time, store_delta], axis = 1)
    
    return store_data   


In [69]:
store_data = data_processing(target_data)
store_data.reset_index(drop = False, inplace = True)
actual_data = store_data.copy()

In [70]:
category_data = pd.get_dummies(store_data['store_name'], prefix = 'store', prefix_sep = '_')

del category_data['store_가덕해안로점']
store_data = pd.concat([store_data, category_data], axis = 1)
store_data.head(3)

,store_name,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,...,store_한누리대로점,store_한밭대로점,store_해안새벽시장길점,store_해운대로점,store_해운대해변로점,store_화곡로2점,store_화곡로점,store_화중로점,store_효덕로점,store_효원로점
0,가덕해안로점,1032,643,188,774,258,771,261,88,100,...,0,0,0,0,0,0,0,0,0,0
1,가마산로2점,1511,939,277,1141,370,1123,388,142,110,...,0,0,0,0,0,0,0,0,0,0
2,가마산로점,1065,648,204,782,283,789,276,103,107,...,0,0,0,0,0,0,0,0,0,0


In [71]:
X_cols_name = 'X_cols.csv'
X_cols = pd.read_csv(os.path.join(model_dir, X_cols_name))
X_cols = X_cols['X_cols']

In [72]:
X = store_data[X_cols].copy()
X.head(3)

,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,order_time_13,...,store_한누리대로점,store_한밭대로점,store_해안새벽시장길점,store_해운대로점,store_해운대해변로점,store_화곡로2점,store_화곡로점,store_화중로점,store_효덕로점,store_효원로점
0,1032,643,188,774,258,771,261,88,100,101,...,0,0,0,0,0,0,0,0,0,0
1,1511,939,277,1141,370,1123,388,142,110,109,...,0,0,0,0,0,0,0,0,0,0
2,1065,648,204,782,283,789,276,103,107,89,...,0,0,0,0,0,0,0,0,0,0


In [73]:
import pickle

model_weekday_name = 'model_y_weekday_GradientBoosting.pickle'
model_weekend_name = 'model_y_weekend_GradientBoosting.pickle'

model_weekday_path = os.path.join(model_dir, model_weekday_name)
model_weekend_path = os.path.join(model_dir, model_weekend_name)

with open(model_weekday_path, mode = 'rb') as f:
    model_weekday = pickle.load(f)
    
with open(model_weekend_path, mode = 'rb') as f:
    model_weekend = pickle.load(f)
    
print(model_weekday)
print(model_weekend)

GradientBoostingClassifier(random_state=0)
GradientBoostingClassifier(random_state=0)


C:\Users\manci\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DummyClassifier from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\manci\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\manci\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator GradientBoostingClassifier from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [74]:
pred_weekday = model_weekday.predict(X)
pred_weekend = model_weekend.predict(X)
pred_weekend[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [75]:
pred_proba_weekday = model_weekday.predict_proba(X)
pred_proba_weekend = model_weekend.predict_proba(X)
pred_proba_weekend[:10]

array([[0.0625097 , 0.9374903 ],
       [0.14219353, 0.85780647],
       [0.10123183, 0.89876817],
       [0.125263  , 0.874737  ],
       [0.05622054, 0.94377946],
       [0.14797686, 0.85202314],
       [0.09442782, 0.90557218],
       [0.08346824, 0.91653176],
       [0.09436701, 0.90563299],
       [0.08788334, 0.91211666]])

In [76]:
pred_proba_weekday = pred_proba_weekday[:,1]
pred_proba_weekend = pred_proba_weekend[:,1]
pred_proba_weekend[:10]

array([0.9374903 , 0.85780647, 0.89876817, 0.874737  , 0.94377946,
       0.85202314, 0.90557218, 0.91653176, 0.90563299, 0.91211666])

In [77]:
pred = pd.DataFrame({'pred_weekday': pred_weekday, 
                     'pred_weekend': pred_weekend, 
                     'score_weekday': pred_proba_weekday, 
                     'score_weekend': pred_proba_weekend})
pred.loc[:, 'store_name'] = store_data['store_name']
pred.loc[:, 'year_month'] = tg_ym
pred.head(3)

,pred_weekday,pred_weekend,score_weekday,score_weekend,store_name,year_month
0,0.0,1.0,0.268458,0.937490,가덕해안로점,202103
1,0.0,1.0,0.207596,0.857806,가마산로2점,202103
2,0.0,1.0,0.413996,0.898768,가마산로점,202103


In [78]:
pred_viz = pred[['store_name', 'score_weekday', 'score_weekend']].copy()
pred_viz.set_index('store_name', inplace = True)
pred_viz

,score_weekday,score_weekend
store_name,,
가덕해안로점,0.268458,0.937490
가마산로2점,0.207596,0.857806
가마산로점,0.413996,0.898768
감천항로점,0.405474,0.874737
강남대로2점,0.307603,0.943779
...,...,...
화곡로2점,0.377046,0.946544
화곡로점,0.216734,0.920831
화중로점,0.253439,0.924777


In [79]:
# 예측 결과 히트맵으로 시각화
import seaborn as sns
import matplotlib as plt

import os

if os.name == 'nt':
    plt.rc('font', family = 'Malgun Gothic')
elif os.name == 'posix':
    plt.rc('font', family = 'AllieGothic')
    
plt.rc('axes', unicode_minus = False)

sns.heatmap(pred_viz[:20].T)

<Axes: xlabel='store_name'>

In [80]:
# 실적 데이터 만들어서 매장용 보고서 만들기(3월 데이터)
target_cols = [
    'store_name', 'order', 'order_fin', 'order_cancel', 'order_delivery', 
    'order_takeout', 'order_weekday', 'order_weekend', 'delta_avg']
actual_data = actual_data[target_cols]
actual_cols = ['store_name']
rename_cols = [f'{x}_{tg_ym}'
               for x in actual_data.columns if x != 'store_name']
actual_cols.extend(rename_cols)
actual_data.columns = actual_cols
actual_data.head(3)

,store_name,order_202103,order_fin_202103,order_cancel_202103,order_delivery_202103,order_takeout_202103,order_weekday_202103,order_weekend_202103,delta_avg_202103
0,가덕해안로점,1032,643,188,774,258,771,261,34.218009
1,가마산로2점,1511,939,277,1141,370,1123,388,34.805511
2,가마산로점,1065,648,204,782,283,789,276,34.499419


In [82]:
# 매장용 보고서 만들기
pred.loc[pred['score_weekday'] >= 0.75, 
         '주문 예측/평일'] = '증가(큼)'
pred.loc[(pred['score_weekday'] < 0.75) & (pred['score_weekday'] >= 0.5), 
         '주문 예측/평일'] = '증가'
pred.loc[(pred['score_weekday'] < 5) & (pred['score_weekday'] >= 0.25),
         '주문 예측/평일'] = '감소'
pred.loc[pred['score_weekday'] < 0.25, 
         '주문 예측/평일'] = '감소(큼)'

pred.loc[pred['score_weekend'] >= 0.75, 
         '주문 예측/휴일'] = '증가(큼)'
pred.loc[(pred['score_weekend'] < 0.75) & (pred['score_weekend'] >= 0.5), 
         '주문 예측/휴일'] = '증가'
pred.loc[(pred['score_weekend'] < 5) & (pred['score_weekend'] >= 0.25),
         '주문 예측/휴일'] = '감소'
pred.loc[pred['score_weekend'] < 0.25, 
         '주문 예측/휴일'] = '감소(큼)'

In [83]:
report = pred[['store_name', '주문 예측/평일', '주문 예측/휴일', 
               'score_weekday', 'score_weekend']]
report = pd.merge(report, actual_data, on = 'store_name', how = 'left')
report.head(3)

,store_name,주문 예측/평일,주문 예측/휴일,score_weekday,score_weekend,order_202103,order_fin_202103,order_cancel_202103,order_delivery_202103,order_takeout_202103,order_weekday_202103,order_weekend_202103,delta_avg_202103
0,가덕해안로점,감소,감소,0.268458,0.937490,1032,643,188,774,258,771,261,34.218009
1,가마산로2점,감소(큼),감소,0.207596,0.857806,1511,939,277,1141,370,1123,388,34.805511
2,가마산로점,감소,감소,0.413996,0.898768,1065,648,204,782,283,789,276,34.499419


In [85]:
pred_ym = datetime.datetime.strptime(tg_ym, '%Y%m')
from dateutil.relativedelta import relativedelta
pred_ym = pred_ym + relativedelta(months = 1)
pred_ym = datetime.datetime.strftime(pred_ym, '%Y%m')

report_name = f'report_pred_{pred_ym}.xlsx'
report.to_excel(os.path.join(output_dir, report_name), index = False)